In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os


folder = r"C:\Users\Rajath\Desktop\Face-Mask-Detection-master\dataset\train"
category = ["mask", "no_mask"]


x = []
y = []

for ca in category:
    path = os.path.join(folder, ca)
    for im in os.listdir(path):
    	im_path = os.path.join(path, im)
    	img = load_img(im_path, target_size=(224, 224))
    	img = img_to_array(img)
    	img = preprocess_input(img)

    	x.append(img)
    	y.append(ca)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
y = lb.fit_transform(y)
y = to_categorical(y)
x = np.array(x, dtype="float32")
y = np.array(y)

(x_train, x_test, y_train, y_test) = train_test_split(x, y,
	test_size=0.20, stratify=y, random_state=42)

# construct the training image generator for data augmentation
augment = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")


bm = MobileNetV2(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))


hm = bm.output
hm = AveragePooling2D(pool_size=(7, 7))(hm)
hm = Flatten(name="flatten")(hm)
hm = Dense(128, activation="relu")(hm)
hm = Dropout(0.5)(hm)
hm = Dense(2, activation="softmax")(hm)


mobile_model = Model(inputs=bm.input, outputs=hm)


for layer in bm.layers:
    layer.trainable = False


optimiser = Adam(lr=0.0001, decay=0.0001 / 20)
mobile_model.compile(loss="binary_crossentropy", optimizer=optimiser,metrics=["accuracy"])


H = mobile_model.fit(augment.flow(x_train, y_train, batch_size=32),steps_per_epoch=len(x_train) // 32,validation_data=(x_test, y_test),validation_steps=len(x_test) // 32,epochs=7)

pred = mobile_model.predict(x_test, batch_size=32)

pred = np.argmax(pred, axis=1)


print(classification_report(y_test.argmax(axis=1), pred,target_names=lb.classes_))

mobile_model.save("mobilenetv2.model", save_format="h5")



E:\anacond3_9\lib\site-packages\PIL\Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


E:\anacond3_9\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/7
116/116 [==============================] - 106s 848ms/step - loss: 0.4641 - accuracy: 0.8063 - val_loss: 0.2075 - val_accuracy: 0.9528
Epoch 2/7
116/116 [==============================] - 105s 905ms/step - loss: 0.1908 - accuracy: 0.9437 - val_loss: 0.1259 - val_accuracy: 0.9678
Epoch 3/7
116/116 [==============================] - 131s 1s/step - loss: 0.1411 - accuracy: 0.9499 - val_loss: 0.0987 - val_accuracy: 0.9689
Epoch 4/7
116/116 [==============================] - 160s 1s/step - loss: 0.1176 - accuracy: 0.9610 - val_loss: 0.0872 - val_accuracy: 0.9743
Epoch 5/7
116/116 [==============================] - 189s 2s/step - loss: 0.0991 - accuracy: 0.9689 - val_loss: 0.0713 - val_accuracy: 0.9839
Epoch 6/7
116/116 [==============================] - 184s 2s/step - loss: 0.0856 - accuracy: 0.9724 - val_loss: 0.0641 - val_accuracy: 0.9839
Epoch 7/7
30/30 [==============================] - 41s 890ms/step
              precision    recall  f1-score   support

        mask       0.

In [6]:
pred = mobile_model.predict(x_test, batch_size=32)
print(pred)

30/30 [==============================] - 26s 870ms/step
[[9.9999857e-01 1.4739564e-06]
 [6.8865402e-06 9.9999309e-01]
 [3.3666131e-01 6.6333872e-01]
 ...
 [1.0923641e-02 9.8907638e-01]
 [6.7912021e-07 9.9999928e-01]
 [9.9999726e-01 2.7443475e-06]]
